<a href="https://colab.research.google.com/github/NIcknameInvalido/star_wars/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação de bibliotecas

In [2]:
import requests as req
import pandas as pd
import re as reg
import sqlite3 as slite

# conexão com o banco de dados SqlLite3
---



In [3]:
conn = slite.connect("db_star_wars")



---



# Consumo das API'S fornecidas pelo  https://swapi.dev/

In [83]:
characters = req.get('https://swapi.dev/api/people')
planets = req.get('https://swapi.dev/api/planets')
films = req.get('https://swapi.dev/api/films')

# Criação de dataframes pandas para manipulação de dados

In [84]:
df_charcter = pd.json_normalize(characters.json(), 'results')
df_planets = pd.json_normalize(planets.json(), 'results')
df_films = pd.json_normalize(films.json(), 'results')

# Tratamentos de dados

### Funções

In [85]:
def extractNumberFromUrl(url):
    found = reg.search(r'/(\d+)/', url)
    return int(found.group(1)) if found else None

### Tratamento do dataframe Characters

In [86]:
df_charcter['characterID'] = df_charcter['url'].apply(extractNumberFromUrl)
df_charcter['planetID'] = df_charcter['homeworld'].apply(extractNumberFromUrl)
df_charcter['filmsID'] = df_charcter['films'].apply(lambda x: [extractNumberFromUrl(url) for url in x])
df_charcter['speciesID'] = df_charcter['species'].apply(lambda x: [extractNumberFromUrl(url) for url in x])
df_charcter['speciesID'] = df_charcter['speciesID'].apply(lambda x:  x[0] if x else 0)
df_charcter['gender'] = df_charcter['gender'].replace('n/a', '')

### Tratamento do dataframe Planets

In [87]:
df_planets['planetID'] = df_planets['url'].apply(extractNumberFromUrl)

### Tratamento do dataframe Films

In [88]:
df_films['filmsID'] = df_films['url'].apply(extractNumberFromUrl)
df_films['characters'] = df_films['characters'].apply(lambda x: [extractNumberFromUrl(url) for url in x])
df_films['planets'] = df_films['planets'].apply(lambda x: [extractNumberFromUrl(url) for url in x])
df_films['vehicles'] = df_films['vehicles'].apply(lambda x: [extractNumberFromUrl(url) for url in x])
df_films['starships'] = df_films['starships'].apply(lambda x: [extractNumberFromUrl(url) for url in x])
df_films['species'] = df_films['species'].apply(lambda x: [extractNumberFromUrl(url) for url in x])

## Visualização de dados

In [89]:
df_charcter[['characterID', 'name', 'planetID', 'speciesID']].head(3)

,characterID,name,planetID,speciesID
0,1,Luke Skywalker,1,0
1,2,C-3PO,1,2
2,3,R2-D2,8,2


In [90]:
df_films.head(2)

,title,episode_id,opening_crawl,director,producer,release_date,characters,planets,starships,vehicles,species,created,edited,url,filmsID
0,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15...","[1, 2, 3]","[2, 3, 5, 9, 10, 11, 12, 13]","[4, 6, 7, 8]","[1, 2, 3, 4, 5]",2014-12-10T14:23:31.880000Z,2014-12-20T19:49:45.256000Z,https://swapi.dev/api/films/1/,1
1,The Empire Strikes Back,5,It is a dark time for the\r\nRebellion. Althou...,Irvin Kershner,"Gary Kurtz, Rick McCallum",1980-05-17,"[1, 2, 3, 4, 5, 10, 13, 14, 18, 20, 21, 22, 23...","[4, 5, 6, 27]","[3, 10, 11, 12, 15, 17, 21, 22, 23]","[8, 14, 16, 18, 19, 20]","[1, 2, 3, 6, 7]",2014-12-12T11:26:24.656000Z,2014-12-15T13:07:53.386000Z,https://swapi.dev/api/films/2/,2


In [91]:
df_planets.head(3)

,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,residents,films,created,edited,url,planetID
0,Tatooine,23,304,10465,arid,1 standard,desert,1,200000,"[https://swapi.dev/api/people/1/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-09T13:50:49.641000Z,2014-12-20T20:58:18.411000Z,https://swapi.dev/api/planets/1/,1
1,Alderaan,24,364,12500,temperate,1 standard,"grasslands, mountains",40,2000000000,"[https://swapi.dev/api/people/5/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T11:35:48.479000Z,2014-12-20T20:58:18.420000Z,https://swapi.dev/api/planets/2/,2
2,Yavin IV,24,4818,10200,"temperate, tropical",1 standard,"jungle, rainforests",8,1000,[],[https://swapi.dev/api/films/1/],2014-12-10T11:37:19.144000Z,2014-12-20T20:58:18.421000Z,https://swapi.dev/api/planets/3/,3


In [92]:
df_charcter_expanded = df_charcter.explode('filmsID')
df_charcter_film = df_charcter_expanded[['characterID', 'filmsID', 'planetID', 'speciesID']]
df_charcter_film = pd.merge(df_charcter_film, df_films, how='inner')


df_charcter_film

In [93]:
df_charcter_film

,characterID,filmsID,planetID,speciesID,title,episode_id,opening_crawl,director,producer,release_date,characters,planets,starships,vehicles,species,created,edited,url
0,1,1,1,0,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15...","[1, 2, 3]","[2, 3, 5, 9, 10, 11, 12, 13]","[4, 6, 7, 8]","[1, 2, 3, 4, 5]",2014-12-10T14:23:31.880000Z,2014-12-20T19:49:45.256000Z,https://swapi.dev/api/films/1/
1,2,1,1,2,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15...","[1, 2, 3]","[2, 3, 5, 9, 10, 11, 12, 13]","[4, 6, 7, 8]","[1, 2, 3, 4, 5]",2014-12-10T14:23:31.880000Z,2014-12-20T19:49:45.256000Z,https://swapi.dev/api/films/1/
2,3,1,8,2,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15...","[1, 2, 3]","[2, 3, 5, 9, 10, 11, 12, 13]","[4, 6, 7, 8]","[1, 2, 3, 4, 5]",2014-12-10T14:23:31.880000Z,2014-12-20T19:49:45.256000Z,https://swapi.dev/api/films/1/
3,4,1,1,0,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15...","[1, 2, 3]","[2, 3, 5, 9, 10, 11, 12, 13]","[4, 6, 7, 8]","[1, 2, 3, 4, 5]",2014-12-10T14:23:31.880000Z,2014-12-20T19:49:45.256000Z,https://swapi.dev/api/films/1/
4,5,1,2,0,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15...","[1, 2, 3]","[2, 3, 5, 9, 10, 11, 12, 13]","[4, 6, 7, 8]","[1, 2, 3, 4, 5]",2014-12-10T14:23:31.880000Z,2014-12-20T19:49:45.256000Z,https://swapi.dev/api/films/1/
5,6,1,1,0,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15...","[1, 2, 3]","[2, 3, 5, 9, 10, 11, 12, 13]","[4, 6, 7, 8]","[1, 2, 3, 4, 5]",2014-12-10T14:23:31.880000Z,2014-12-20T19:49:45.256000Z,https://swapi.dev/api/films/1/
6,7,1,1,0,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15...","[1, 2, 3]","[2, 3, 5, 9, 10, 11, 12, 13]","[4, 6, 7, 8]","[1, 2, 3, 4, 5]",2014-12-10T14:23:31.880000Z,2014-12-20T19:49:45.256000Z,https://swapi.dev/api/films/1/
7,8,1,1,2,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15...","[1, 2, 3]","[2, 3, 5, 9, 10, 11, 12, 13]","[4, 6, 7, 8]","[1, 2, 3, 4, 5]",2014-12-10T14:23:31.880000Z,2014-12-20T19:49:45.256000Z,https://swapi.dev/api/films/1/
8,9,1,1,0,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15...","[1, 2, 3]","[2, 3, 5, 9, 10, 11, 12, 13]","[4, 6, 7, 8]","[1, 2, 3, 4, 5]",2014-12-10T14:23:31.880000Z,2014-12-20T19:49:45.256000Z,https://swapi.dev/api/films/1/
9,10,1,20,0,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15...","[1, 2, 3]","[2, 3, 5, 9, 10, 11, 12, 13]","[4, 6, 7, 8]","[1, 2, 3, 4, 5]",2014-12-10T14:23:31.880000Z,2014-12-20T19:49:45.256000Z,https://swapi.dev/api/films/1/


# **Banco de dados**

---



## Deleção das tabelas para novas inserções

In [14]:

conn.execute("""
    DROP TABLE IF EXISTS dim_planets
""")
conn.execute("""
    DROP TABLE IF EXISTS fact_films
""")
conn.execute("""
    DROP TABLE IF EXISTS dim_characters
""")
conn.commit()

## Criação das tabelas para armazenar os dados

In [15]:
conn.execute("""
    CREATE TABLE IF NOT EXISTS planets(
        planetID INTEGER NOT NULL PRIMARY KEY,
        name VARCHAR(255) NOT NULL,
        rotation_period INTEGER NOT NULL,
        orbital_period INTEGER,
        diameter INTEGER,
        climate VARCHAR(15),
        gravity VARCHAR(30),
        terrain VARCHAR(60),
        surface_water INTEGER
    )
""")


conn.execute("""
    CREATE TABLE IF NOT EXISTS films(
        filmID INTEGER NOT NULL PRIMARY KEY,
        title VARCHAR(255),
        release_date DATE
    )
""")

conn.execute("""
    CREATE TABLE IF NOT EXISTS characters(
        characterID INTEGER NOT NULL PRIMARY KEY,
        name VARCHAR(255) NOT NULL,
        height INTEGER,
        mass INTEGER,
        hair_color VARCHAR(15),
        skin_color VARCHAR(15),
        eye_color VARCHAR(15),
        birth_year VARCHAR(30),
        gender VARCHAR(15)
    )
""")

conn.commit()

## Inserção de registros

In [16]:
data_characters = df_charcter.to_dict(orient='records')
data_films = df_films.to_dict(orient='records')
data_planets = df_planets.to_dict(orient='records')

In [17]:

for films in data_films:
    conn.execute(
        """INSERT INTO films (filmID, title, release_date) VALUES (?, ?, ?) """,
            (
                films['filmsID'],
                films['title'],
                films['release_date'],
            )
    )
    conn.commit()

for planet in data_planets:
    conn.execute(
        """INSERT INTO planets (planetID, name, rotation_period, orbital_period, diameter, climate, gravity, terrain, surface_water) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?) """,
            (
                planet['planetID'],
                planet['name'],
                planet['rotation_period'],
                planet['orbital_period'],
                planet['diameter'],
                planet['climate'],
                planet['gravity'],
                planet['terrain'],
                planet['surface_water']
            )
    )
    conn.commit()




for character in data_characters:
    conn.execute(
        """INSERT INTO characters (characterID, name, height, mass, hair_color, skin_color, eye_color, birth_year, gender) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?) """,
            (
                character['characterID'],
                character['name'],
                character['height'],
                character['mass'],
                character['hair_color'],
                character['skin_color'],
                character['eye_color'],
                character['birth_year'],
                character['gender']
            )
    )
    conn.commit()